In [12]:
# all imports
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
from io import BytesIO
!pip -q install pydub
from pydub import AudioSegment

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  #with open('audioRecord.wav','wb') as f:
    #f.write(b)
  #return 'audioRecord.wav'  # or webm ?
  audio = AudioSegment.from_file(BytesIO(b))
  return audio

In [15]:
audio = record()
audio

<IPython.core.display.Javascript object>

<class 'pydub.audio_segment.AudioSegment'>


In [2]:
#!apt install ffmpeg
#!pip install git+https://github.com/openai/whisper.git
#!pip install openai
! pip install git+https://github.com/openai/whisper.git
! pip install jiwer

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-xpytpzju
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-xpytpzju
  Resolved https://github.com/openai/whisper.git to commit e8622f9afc4eba139bf796c210f5c01081000472
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.9 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=798395 sha256=9065bcefa63d5c6bab42d7bf0d9ad9088031f7a2fa29e7c809ebfd11e77fb6f5
  Stored in directory: /tmp/pip-ephem-wheel-cache-lhkc1rx5/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.6 MB/s eta 0:00:00


In [4]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import whisper
import torchaudio

from tqdm.notebook import tqdm

In [5]:
from google.colab import drive

# Mount Google Drive
drive.mount("/content/drive", force_remount=True)


Mounted at /content/drive


In [6]:
%cd /content/drive/MyDrive/Tesi/my_dataset/Actor_01/
#%cd /content/drive/MyDrive/Tesi/testtraduzioneSTT/
!ls

/content/drive/MyDrive/Tesi/my_dataset/Actor_01
03-01-01-01-01-01-01.wav  03-01-04-01-01-01-01.wav  03-01-06-02-01-01-01.wav
03-01-01-01-01-02-01.wav  03-01-04-01-01-02-01.wav  03-01-06-02-01-02-01.wav
03-01-01-01-02-01-01.wav  03-01-04-01-02-01-01.wav  03-01-06-02-02-01-01.wav
03-01-01-01-02-02-01.wav  03-01-04-01-02-02-01.wav  03-01-06-02-02-02-01.wav
03-01-02-01-01-01-01.wav  03-01-04-02-01-01-01.wav  03-01-07-01-01-01-01.wav
03-01-02-01-01-02-01.wav  03-01-04-02-01-02-01.wav  03-01-07-01-01-02-01.wav
03-01-02-01-02-01-01.wav  03-01-04-02-02-01-01.wav  03-01-07-01-02-01-01.wav
03-01-02-01-02-02-01.wav  03-01-04-02-02-02-01.wav  03-01-07-01-02-02-01.wav
03-01-02-02-01-01-01.wav  03-01-05-01-01-01-01.wav  03-01-07-02-01-01-01.wav
03-01-02-02-01-02-01.wav  03-01-05-01-01-02-01.wav  03-01-07-02-01-02-01.wav
03-01-02-02-02-01-01.wav  03-01-05-01-02-01-01.wav  03-01-07-02-02-01-01.wav
03-01-02-02-02-02-01.wav  03-01-05-01-02-02-01.wav  03-01-07-02-02-02-01.wav
03-01-03-01-01-01-01.wav  03

In [14]:
model = whisper.load_model("base")

# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("03-01-01-01-01-01-01.wav")
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# decode the audio
#options = whisper.DecodingOptions()
options = whisper.DecodingOptions(fp16 = False)
result = whisper.decode(model, mel, options)

# print the recognized text
print(result.text)

Detected language: it
Dove sono andati ragazzi?


In [1]:
!pip install gtts

In [17]:
!ls

03-01-01-01-01-01-01.wav  03-01-04-01-01-01-01.wav  03-01-06-02-01-01-01.wav
03-01-01-01-01-02-01.wav  03-01-04-01-01-02-01.wav  03-01-06-02-01-02-01.wav
03-01-01-01-02-01-01.wav  03-01-04-01-02-01-01.wav  03-01-06-02-02-01-01.wav
03-01-01-01-02-02-01.wav  03-01-04-01-02-02-01.wav  03-01-06-02-02-02-01.wav
03-01-02-01-01-01-01.wav  03-01-04-02-01-01-01.wav  03-01-07-01-01-01-01.wav
03-01-02-01-01-02-01.wav  03-01-04-02-01-02-01.wav  03-01-07-01-01-02-01.wav
03-01-02-01-02-01-01.wav  03-01-04-02-02-01-01.wav  03-01-07-01-02-01-01.wav
03-01-02-01-02-02-01.wav  03-01-04-02-02-02-01.wav  03-01-07-01-02-02-01.wav
03-01-02-02-01-01-01.wav  03-01-05-01-01-01-01.wav  03-01-07-02-01-01-01.wav
03-01-02-02-01-02-01.wav  03-01-05-01-01-02-01.wav  03-01-07-02-01-02-01.wav
03-01-02-02-02-01-01.wav  03-01-05-01-02-01-01.wav  03-01-07-02-02-01-01.wav
03-01-02-02-02-02-01.wav  03-01-05-01-02-02-01.wav  03-01-07-02-02-02-01.wav
03-01-03-01-01-01-01.wav  03-01-05-02-01-01-01.wav  03-01-08-01-01-01-01.wav

In [33]:
from gtts import gTTS
from io import BytesIO
from tempfile import TemporaryFile
import librosa
import IPython.display as ipd

text = 'L\'Italia ha vinto l\'ultimo mondiale'
tts = gTTS(text=text, lang='it')
f = TemporaryFile()
tts.write_to_fp(f)

f.seek(0)
audio, sr = librosa.load(f)
ipd.Audio(audio, rate=sr)
